###### @author: Phongphat Wiwatthanasetthakarn   @create: 2020-04-07

In [1]:
pip install google-cloud-translate

  Using cached google_cloud_translate-2.0.1-py2.py3-none-any.whl (90 kB)
  Using cached google_api_core-1.17.0-py2.py3-none-any.whl (70 kB)
  Using cached google_cloud_core-1.3.0-py2.py3-none-any.whl (26 kB)
Note: you may need to restart the kernel to use updated packages.  Downloading protobuf-3.12.0-cp37-cp37m-win_amd64.whl (1.0 MB)
  Using cached googleapis-common-protos-1.51.0.tar.gz (35 kB)

  Using cached rsa-4.0-py2.py3-none-any.whl (38 kB)
  Using cached cachetools-4.1.0-py3-none-any.whl (10 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Created wheel for googleapis-common-protos: filename=googleapis_common_protos-1.51.0-py3-none-any.whl size=77602 sha256=c4cd5279905b311708d622a7716e2aed6861c208358c7e9a5bd9b2bc679e8555
  Stored in directory: c:\users\os\appdata\local\pip\cache\wheels\4c\a1\71\5e427276ceeff277fd76878d1b19fbf4587a2845015d86864b
Successfully built googleapis-common-protos


In [2]:
pip install tqdm --upgrade

Requirement already up-to-date: tqdm in c:\programfiles_anaconda\anaconda3\lib\site-packages (4.46.0)
Note: you may need to restart the kernel to use updated packages.


In [23]:
#web scraping
import requests
from bs4 import BeautifulSoup
from time import time, sleep
from random import randint

#Translation
from google.cloud import translate

#Utilities
from tqdm import tqdm
import numpy as np
import pandas as pd

In [24]:
scraping_path = "./scraping/"

In [25]:
def honestdoc_comment(hosp_name):
	"""
	This function is to scrap data from a webboard (https://www.honestdocs.com).
	"""
	#create connection
	url = r"https://www.honestdocs.co/hospitals/"
	url += hosp_name
	data = requests.get(url)
	print("requests code : {}".format(data.status_code)) 
	print("note\n2xx: success\n4xx, 5xx: error")

	#scrape comment and score
	start_time = time() #start scraping data from page1
	r = requests.get(url, params=dict(query="web scraping",page=1)) 
	soup = BeautifulSoup(r.text,"html.parser")
	n = len(soup.find_all("div",{"class":"comments__content"})) #find n of items in the page

	#extract each item
	comment = [soup.find_all("div",
							 {"class":"comments__content"})[i].get_text().strip() for i in range(0,n)]
	score = [soup.find_all("span",
						   {"class":"stars star-rating"})[i].attrs["data-score"] for i in range(0,n)]
	elapsed_time = time() - start_time #finish scraping data from page1
	print("Time used for scraping data from page - 1 : {} s".format(elapsed_time))
	sleep(randint(1,3)) #mimic human behavior
		   
	p = 2 #start scraping data from page2
	while n > 0: #until the number of items in a page = 0
		start_time = time() 
		r = requests.get(url, params=dict(query="web scraping",page=p))
		soup = BeautifulSoup(r.text,"html.parser")
		n = len(soup.find_all("div",{"class":"comments__content"}))
		[comment.append(soup.find_all("div",
									  {"class":"comments__content"})[i].get_text().strip()) for i in range(0,n)]
		[score.append(soup.find_all("span",
									{"class":"stars star-rating"})[i].attrs["data-score"]) for i in range(0,n)]
		elapsed_time = time() - start_time
		print("Time used for scraping data from page - {} : {} s".format(p, elapsed_time))
		p +=1
		sleep(randint(1,3))


	#backup data 
	pd.DataFrame({"comment": comment,
				  "score": score
				}).to_csv(scraping_path + "comment_" + hosp_name + ".csv", index=False)	  

	return comment, score
		

In [26]:
#Credential json file format that generated from google cloud platform service
import os
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"c:\Users\os\Desktop\RADS611\Lab\sentimenthospital.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"./credential/sentimenthospital.json"

In [27]:
#Project id from google cloud platform service
project_id="sentimenthospital"


def translate_text(text):
    client = translate.TranslationServiceClient()
    parent = client.location_path(project_id, "global")

    response = client.translate_text(
        parent=parent,
        contents=[text],
        mime_type="text/plain",  # mime types: text/plain, text/html
        source_language_code="th",
        target_language_code="en",
    )    
    
    for translation in response.translations:        
        text_return = translation.translated_text
    
    return text_return

In [31]:
hosp_name_list = ["ramathibodi-hospital", "siriraj-hospital", "king-chulalongkorn-memorial-hospital", "bangkok-hospital"]
#
tqdm.pandas()

c:\ProgramFiles_Anaconda\Anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [32]:
for i in range(len(hosp_name_list)):
    #
    file_name = scraping_path + "comment_" + hosp_name_list[i] + ".csv"
    file_name_en = scraping_path + "comment_" + hosp_name_list[i] + "_en" + ".csv"
    #
    print("\n =================")
    print("\n hospital name = " + hosp_name_list[i])
    print("\n =================")
    #
    comments, scores = honestdoc_comment(hosp_name_list[i])
    #
    com_df = pd.read_csv(file_name)
    #    
    com_df["en"] = com_df.progress_apply( lambda x: translate_text(x["comment"]), axis=1  )
    #
    print("\nPreview data\n")
    print(com_df)
    #
    com_df.to_csv(file_name_en, index= False)



 hospital name = ramathibodi-hospital

requests code : 200
note
2xx: success
4xx, 5xx: error
Time used for scraping data from page - 1 : 0.36368846893310547 s
Time used for scraping data from page - 2 : 0.37401247024536133 s
Time used for scraping data from page - 3 : 0.3660440444946289 s
Time used for scraping data from page - 4 : 0.36528992652893066 s
Time used for scraping data from page - 5 : 0.3946206569671631 s
Time used for scraping data from page - 6 : 0.3676881790161133 s
Time used for scraping data from page - 7 : 0.3675520420074463 s
Time used for scraping data from page - 8 : 0.39696192741394043 s
Time used for scraping data from page - 9 : 0.36362171173095703 s
Time used for scraping data from page - 10 : 0.3807680606842041 s
Time used for scraping data from page - 11 : 0.40509676933288574 s
Time used for scraping data from page - 12 : 0.35715794563293457 s
Time used for scraping data from page - 13 : 0.5846250057220459 s
Time used for scraping data from page - 14 : 0.38

100%|██████████| 224/224 [02:13<00:00,  1.68it/s]



Preview data

                                               comment  score  \
0    เคยได้ไปลองใช้บริการคลีนิคนอกเวลาของรามาเมื่อต...      5   
1    ผมเคยไปทำเลสิกที่นี้ ดีมากครับตอนนี้ผ่ามาจะ2ปี...      4   
2    รักษามาหลายปีแล้วตั้งแต่อายุ14 ตอนนี้จะจบปริญญ...      5   
3    ที่ตึกพระเทพชั้น 2 สถานที่เครื่องมือดูทันสมัย ...      3   
4    Its a public hospital so service is bad (nurse...      4   
..                                                 ...    ...   
219  การบริการดี รพ สะอาดหมอสอบถามตลอดคอยดุแลตลอดคอ...      5   
220                                        บริการดีมาก      5   
221  คนไข้ค่อนข้างเยอะทำให้การได้รับการบริการค่อนข่...      4   
222       คอยตั้งแต่9โมงเช้าถึงเที่ยงไม่ได้ระบการรักษา      1   
223  คุณหมอเก่ง ให้คำแนะนำที่ละเอียดชัดเจนดี การผ่า...      5   

                                                    en  
0    Had tried the part-time clinic service at the ...  
1    I've been to Lasik here. Very good. Now, after...  
2    Healed many years since age 

100%|██████████| 132/132 [01:17<00:00,  1.70it/s]



Preview data

                                               comment  score  \
0    พยาบาลบริการดีคุณหมอสุภาพโรงพยาบาลรักษาความสะอ...      5   
1    ใช้บริการปรึกษาเรื่องแผลคีลอยด์ที่บริเวรต้นแขน...      5   
2    เป็นโรงพยาบาลที่ดีมากๆคะเคยรักษาเนื้องอกที่มดล...      5   
3    รพ.ศิริราช คุณหมอและพยาบาลน่ารักมากค่ะ ให้คำแน...      5   
4    พาคุณย่าไปรักษาโรคมะเร็ง บริการดี รวดเร็วเป็นก...      5   
..                                                 ...    ...   
127  เริ่มแรกเข้า ER เนื่องจากคุณแม่ล้ม และไข้ขึ้นส...      5   
128  ผมได้รับการรักษามะเร็งจนหายดีใช้ชีวิตเหมือนคนป...      5   
129  เนื่องจากการรักษาต้องทำการผ่าตัด และฉันเลือกดม...      3   
130                                     บริการก็ใช้ได้      4   
131  สถานที่คับแคบเมื่อเทียบกับจำนวนคนที่ใช้บริการ ...      4   

                                                    en  
0    The nurse is very good, the doctor is polite, ...  
1    Use a consultative service for keloids on the ...  
2    It is a very good hospital. 

Time used for scraping data from page - 100 : 0.3927161693572998 s
Time used for scraping data from page - 101 : 0.3721473217010498 s
Time used for scraping data from page - 102 : 0.3464512825012207 s
Time used for scraping data from page - 103 : 0.36713624000549316 s
Time used for scraping data from page - 104 : 0.3723886013031006 s
Time used for scraping data from page - 105 : 0.40877485275268555 s
Time used for scraping data from page - 106 : 0.37152624130249023 s
Time used for scraping data from page - 107 : 0.3447589874267578 s
Time used for scraping data from page - 108 : 0.33499741554260254 s
Time used for scraping data from page - 109 : 0.34685349464416504 s
Time used for scraping data from page - 110 : 0.4071919918060303 s
Time used for scraping data from page - 111 : 0.36176466941833496 s
Time used for scraping data from page - 112 : 0.37180161476135254 s
Time used for scraping data from page - 113 : 0.36124706268310547 s
Time used for scraping data from page - 114 : 0.361190

100%|██████████| 355/355 [03:26<00:00,  1.72it/s]



Preview data

                                               comment  score  \
0    เรื่อง ของเรื่องมีอยู่ว่า ไม่มีไข้ แต่โดนกักตั...      5   
1    บริการค่อนข้างดีค่ะ คุณหมอพยาบาลใจดีเป็นกันเอง...      4   
2    ไปรับการตรวจเรื่องเม็ดเลือดในปัสสาวะค่ะ ต้องจอ...      4   
3    บริการค่อยข้างดีค่ะ แต่รอคิวนานนิดหน่อย แต่วิน...      5   
4    ได้มีอาการอาหารเป็นพิษและเข้าทำการรักษาที่โรงพ...      5   
..                                                 ...    ...   
350  ค่ำวันที่ 26 ตุลาคม 2559 เหตุการณ์ในวันนั้นฉัน...      5   
351  ทำเพื่ออะไรรีวิวนี้แต่เราไปรักษารพจุฬาก็ได้รับ...      5   
352  ทาง รพ. บริการคนไข้ดีมาก ใส่ใจผู้ป่วย เป็นกันเ...      5   
353                                            หมอเก่ง      3   
354  เมื่อวันที่ 7 เมษายน 2559 ตัวผมเองรุ้สึกว่าไม่...      4   

                                                    en  
0    The matter is that there is no fever but being...  
1    The service is quite good. The doctor was very...  
2    Went to get a blood test in 

100%|██████████| 241/241 [02:15<00:00,  1.78it/s]


Preview data

                                               comment  score  \
0    คุณหมอที่แผนกอายุรกรรมเป็นกันเอง วินิจฉัยถูกโร...      4   
1    เคยไปรักษาตัวอยู่ที่แผนกจิตรักษ์ แผนกดูแลเรื่อ...      5   
2    บริการดีมาก รวดเร็ว พาแขกคนจีนไปก็มีล่ามจีนแปล...      3   
3    มีการบริการดี คุณหมอมีความรู้ความสามารถมาก และ...      4   
4    บริการดีมากชัดเจนดีพยาบาลก็พูดดีบุคลากรพูดดีหม...      5   
..                                                 ...    ...   
236  โรงพยาบาลสะอาด /การดูแลของแพทย์และเอาใจใส่เป็น...      5   
237  เกิดที่รพ..นี้ จนตอนนี้ 30 กว่าปีคุณหมอหรือพยา...      5   
238              สะดวกใส่ใจทุกรายบะเอียดบริการเป็นเลิศ      5   
239  วันนี้ไปหาหมอที่รพ.นี้ เป็นรพ.ที่ดีมาก เพราะที...      5   
240  เคยเข้าไปใช้บริการอยู่ครั้งสองครั้งค่ะ ดิฉันจำ...      3   

                                                    en  
0    The doctor at the Internal Medicine Department...  
1    Used to heal himself at the Conservation Depar...  
2    Service is very good, fast, 